# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [2]:
import numpy as np


#WOLF ACTIONS
def up(position):
    if(position <= 2):
        return position + 2
    else:
        return position - 2
    
def down(position):
    if(position > 2):
        return position - 2
    else:
        return position + 2

def right(position):
    if(position == 1 or position ==3):
        return position + 1
    else:
        return position -1
    
def left(position):
    if(position == 2 or position == 4):
        return position - 1
    else:
        return position + 1
    
def stay(position):
    return position

#HARE MOVEMENTS

def hare_movements(position, actions):
    positions = []
    for action in actions:
        positions = positions + [action(position)]
    return list(set(positions))



hare_stay = 0.6
hare_move = 0.4
hare_choice = 0.5

wolf_fail = 0.2
wolf_success = 0.8
wolf_stay = 1


state_space =[(1,1),(1,2),(1,3),(1,4),(2,1),(2,2),(2,3),(2,4),(3,1),(3,2),(3,3),(3,4),(4,1),(4,2),(4,3),(4,4)]

action_space = [up, down, left, right, stay]

m_dictionary = {(1,1):0, (1,2):1 ,(1,3):2, (1,4):3, (2,1):4, (2,2):5,
                (2,3):6, (2,4):7, (3,1):8, (3,2):9, (3,3):10, (3,4):11
                ,(4,1):12, (4,2):13, (4,3):14, (4,4):15}






def gen_cost_matrix(states, actions):
    C = np.zeros(shape = (len(states), len(actions)))
    C[:] = 1
    equality = [(1,1),(2,2),(3,3),(4,4)]
    verticality = [(1,3),(3,1),(2,4),(4,2)]
    horizontality = [(1,2),(2,1),(3,4),(4,3)]
    diagonality = [(1,4),(4,1),(2,3),(3,2)]
            
    
    for state in states:
        if state in equality:
            C[m_dictionary[state]][4] = 0
        elif state in verticality:
            C[m_dictionary[state]][0] = 0
            C[m_dictionary[state]][1] = 0
        elif state in horizontality:
            C[m_dictionary[state]][2] = 0
            C[m_dictionary[state]][3] = 0
        elif state in diagonality:
            for action in range(0,len(actions)-1):
                C[m_dictionary[state]][action] = 0.5
        
    
    return C

Hare = np.array([[0.6, 0.2, 0.2, 0],
                 [0.2, 0.6, 0, 0.2],
                 [0.2, 0, 0.6, 0.2], 
                 [0, 0.2, 0.2, 0.6]])


Wup = Wdown = np.array([[0.2, 0, 0.8, 0],
                        [0, 0.2, 0, 0.8],
                        [0.8, 0, 0.2, 0],
                        [0, 0.8, 0, 0.2]])

Wleft = Wright = np.array([[0.2, 0.8, 0, 0],
                           [0.8, 0.2, 0, 0],
                           [0, 0, 0.2, 0.8],
                           [0, 0, 0.8, 0.2]])

Wstay = ([[1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]])



np.set_printoptions(linewidth = 200)

Pup = np.kron(Wup, Hare)
Pdown = np.kron(Wdown, Hare)
Pright = np.kron(Wright, Hare)
Pleft = np.kron(Wleft, Hare)
Pstay = np.kron(Wstay, Hare)

Cost = gen_cost_matrix(state_space, action_space)

print("Cost= \n",Cost)


Cost= 
 [[ 1.   1.   1.   1.   0. ]
 [ 1.   1.   0.   0.   1. ]
 [ 0.   0.   1.   1.   1. ]
 [ 0.5  0.5  0.5  0.5  1. ]
 [ 1.   1.   0.   0.   1. ]
 [ 1.   1.   1.   1.   0. ]
 [ 0.5  0.5  0.5  0.5  1. ]
 [ 0.   0.   1.   1.   1. ]
 [ 0.   0.   1.   1.   1. ]
 [ 0.5  0.5  0.5  0.5  1. ]
 [ 1.   1.   1.   1.   0. ]
 [ 1.   1.   0.   0.   1. ]
 [ 0.5  0.5  0.5  0.5  1. ]
 [ 0.   0.   1.   1.   1. ]
 [ 1.   1.   0.   0.   1. ]
 [ 1.   1.   1.   1.   0. ]]


### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [3]:
def gen_policy_matrix(states, actions):
    Pi = np.zeros(shape = (len(states), len(actions)))
    Pi[:] = 0
    equality = [(1,1),(2,2),(3,3),(4,4)]
    verticality = [(1,3),(3,1),(2,4),(4,2)]
    horizontality = [(1,2),(2,1),(3,4),(4,3)]
    diagonality = [(1,4),(4,1),(2,3),(3,2)]
            
    
    for state in states:
        if state in equality:
            Pi[m_dictionary[state]][4] = 1
        elif state in verticality:
            Pi[m_dictionary[state]][0] = 0.5
            Pi[m_dictionary[state]][1] = 0.5
        elif state in horizontality:
            Pi[m_dictionary[state]][2] = 0.5
            Pi[m_dictionary[state]][3] = 0.5
        elif state in diagonality:
            for action in range(0,len(actions)-1):
                Pi[m_dictionary[state]][action] = 0.25
        
    
    return Pi


Pi = gen_policy_matrix(state_space, action_space)
print("Pi= \n", Pi)


Pi= 
 [[ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [4]:
p_matrices = [Pup, Pdown, Pleft, Pright, Pstay]

def generate_P_Pi(policy_matrix, probability_matrices):
    Ppi = np.zeros(shape = (16, 16))
    for x in range(np.shape(policy_matrix)[0]):
        for y in range(np.shape(policy_matrix)[1]):
            Ppi[x] =  Ppi[x] + np.multiply (policy_matrix[x][y], (probability_matrices[y])[x])
    return Ppi


def generate_C_Pi(policy_matrix, cost_matrix):
    Cpi = np.zeros(shape = (16))
    for x in range(np.shape(policy_matrix)[0]):
        for y in range(np.shape(policy_matrix)[1]):
            Cpi[x] =  Cpi[x] + np.multiply (policy_matrix[x][y], cost_matrix[x][y])
    return Cpi


def compute_cost_to_go(Ppi, Cpi, discount):
    I = np.identity(Ppi.shape[0])
    scaled_p = np.multiply(discount, Ppi)
    subtraction = np.subtract(I, scaled_p)
    inversion = np.linalg.inv(subtraction)
    J =np.dot(inversion, Cpi)
    return J
        
    
    
Ppi = generate_P_Pi(Pi, p_matrices)        
Cpi = generate_C_Pi(Pi, Cost)

J = compute_cost_to_go(Ppi, Cpi, 0.99)

print("J =\n",J)

J =
 [ 1.13980941  1.16859248  1.16859248  1.83787144  1.16859248  1.13980941  1.83787144  1.16859248  1.16859248  1.83787144  1.13980941  1.16859248  1.83787144  1.16859248  1.16859248  1.13980941]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [6]:
import time as time
def value_iteration(cost_matrix, discount):
    i = 0
    err = 1
    J = np.zeros(16)
    start = time.time()
    while err > 1e-8:
        Qup = cost_matrix[:,0] + np.multiply(discount, np.dot(Pup, J))
        Qdown = cost_matrix[:,1] + np.multiply(discount, np.dot(Pdown, J))
        Qleft = cost_matrix[:,2] + np.multiply(discount, np.dot(Pleft, J))
        Qright = cost_matrix[:,3] + np.multiply(discount, np.dot(Pright, J))
        Qstay = cost_matrix[:,4] + np.multiply(discount, np.dot(Pstay, J))
        
        Jnew = np.min((Qup, Qdown, Qleft, Qright, Qstay), axis = 0)
        err = np.linalg.norm(Jnew - J)
        i +=1
        
        J = Jnew
    end = time.time()
    return J, i, end - start


J_value_iter, iterations, performance = value_iteration(Cost, 0.99)
print("Optimal J =")
print(J_value_iter, "\n")
print("Calculated in :", iterations, "iterations for", performance, "seconds")


Optimal J =
[ 1.13980917  1.16859223  1.16859223  1.8378712   1.16859223  1.13980917  1.8378712   1.16859223  1.16859223  1.8378712   1.13980917  1.16859223  1.8378712   1.16859223  1.16859223  1.13980917] 

Calculated in : 1530 iterations for 0.06505537033081055 seconds


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [ ]:
def policy_iteration(Ppi, Cpi, cost_matrix, discount):
    Pi = np.ones((16, 5))
    i = 0
    finished = False
    start = time.time()
    while not finished:
        J = compute_cost_to_go(Ppi, Cpi, 0.99)
        
        Qup = (cost_matrix[:,0] + np.multiply(discount, np.dot(Pup, J))).reshape(16,1)
        Qdown = (cost_matrix[:,1] + np.multiply(discount, np.dot(Pdown, J))).reshape(16,1)
        Qleft = (cost_matrix[:,2] + np.multiply(discount, np.dot(Pleft, J))).reshape(16,1)
        Qright = (cost_matrix[:,3] + np.multiply(discount, np.dot(Pright, J))).reshape(16,1)
        Qstay = (cost_matrix[:,4] + np.multiply(discount, np.dot(Pstay, J))).reshape(16,1)
        
        Pinew = np.zeros(shape = (16, 5))
        
        Pinew[:, 0, None] = np.isclose(Qup, np.min([Qup, Qdown, Qleft, Qright, Qstay], axis = 0), 
                                       atol = 1e-10, rtol = 1e-10).astype(int)
        Pinew[:, 1, None] = np.isclose(Qdown, np.min([Qup, Qdown, Qleft, Qright, Qstay], axis = 0), 
                                       atol = 1e-10, rtol = 1e-10).astype(int)
        Pinew[:, 2, None] = np.isclose(Qleft, np.min([Qup, Qdown, Qleft, Qright, Qstay], axis = 0), 
                                       atol = 1e-10, rtol = 1e-10).astype(int)
        Pinew[:, 3, None] = np.isclose(Qright, np.min([Qup, Qdown, Qleft, Qright, Qstay], axis = 0), 
                                       atol = 1e-10, rtol = 1e-10).astype(int)
        Pinew[:, 4, None] = np.isclose(Qstay, np.min([Qup, Qdown, Qleft, Qright, Qstay], axis = 0), 
                                       atol = 1e-10, rtol = 1e-10).astype(int)
        
        Pinew = Pinew / np.sum(Pinew, axis = 1, keepdims = True)
        
        finished = (Pi == Pinew).all()
        
        Pi = Pinew
        i +=1
        
    end = time.time()
    return Pi, i, end - start

Pi_Optimal, iters, perf = policy_iteration(Ppi, Cpi, Cost, 0.99)
print("Optimal Pi =")
print(Pi_Optimal, "\n")
print("Calculated in :", iters, "iterations for", perf, "seconds")

### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [ ]:
a = (1, 4)
b = (1, 3)

a_trajectories = []
b_trajectories = []

state_map = np.arange(16)
action_map = np.arange(5)

action_matrices = [Pup, Pdown, Pleft, Pright, Pstay]
action_3d_matrix = np.zeros((5, 16, 16))

for action in action_map:
    action_3d_matrix[action] = action_matrices[action]

a_cost = 0
b_cost = 0

for i in range(100):
    gamma = 1

    
    a_trajectory = [a]
    b_trajectory = [b]

    for i in range(9999):

        a_begin = m_dictionary[a_trajectory[-1]]
        b_begin = m_dictionary[b_trajectory[-1]]
        
        
        a_action = np.random.choice(action_map, size = 1,p = Pi_Optimal[a_begin])
        b_action = np.random.choice(action_map, size = 1,p = Pi_Optimal[b_begin])
        
        a_action = a_action[0]
        b_action = b_action[0]
        
        
        a_p = action_3d_matrix[a_action][a_begin] 
        b_p = action_3d_matrix[b_action][b_begin] 
        
        next_a_state = np.random.choice(state_map, size = 1,
                                        p = a_p)[0]
        
        next_b_state = np.random.choice(state_map, size = 1,p = b_p)[0]
        
        a_trajectory += [state_space[next_a_state]]
        b_trajectory += [state_space[next_b_state]]
        
        a_cost += Cost[a_begin][a_action] * gamma
        b_cost += Cost[b_begin][b_action] * gamma
        
        gamma = gamma * 0.99
        
    a_trajectories += [a_trajectory]
    b_trajectories += [b_trajectory]
    
    
    
print("(1,4) a_cost =", a_cost/100)
print("(1,3) b_cost =", b_cost/100)
    
"""
Como esperado, os valores obtidos através da simulação são bastante semelhantes aos obtidos 
anteriormente no cálculo do J*, para os estados correspondentes
"""
    